In [1]:
import pandas as pd
import datetime
from typing import List
import pyecharts.options as opts
from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Timeline, Grid, Bar, Map, Pie, Line

In [2]:
# 显示所有列(参数设置为None代表显示所有行，也可以自行设置数字)
pd.set_option('display.max_columns', None)
# 显示所有行
pd.set_option('display.max_rows', None)
# 设置数据的显示长度，默认为50
pd.set_option('max_colwidth', 200)
# 禁止自动换行(设置为False不自动换行，True反之)
pd.set_option('expand_frame_repr', False)

In [50]:
#url = '../data/DXYArea.csv'
url = 'https://raw.githubusercontent.com/BlankerL/DXY-2019-nCoV-Data/master/csv/DXYArea.csv'
df = pd.read_table(url, sep=',')

In [51]:
provinceName = ["上海", "云南", "内蒙古", "北京", "吉林", "四川",
                "天津", "宁夏", "安徽", "山东", "山西", "广东",
                "广西", "新疆", "江苏", "江西", "河北", "河南",
                "浙江", "海南", "湖北", "湖南", "甘肃", "福建",
                "西藏", "贵州", "辽宁", "重庆", "陕西", "青海", "黑龙江"]

# 格式化DataFrame的日期
df.drop(['cityName', 'city_confirmedCount', 'city_suspectedCount',
         'city_curedCount', 'city_deadCount'], inplace=True, axis=1)
df['updateTime'] = pd.to_datetime(df['updateTime'])
df['updateTime'] = df['updateTime'].apply(
    lambda x: datetime.datetime.strftime(x, '%m-%d'))
#print(df)
df['provinceName'] = df['provinceName'].apply(
    lambda y: y[:3] if y == '内蒙古自治区' or y == '黑龙江省' else y[:2])

# 数据去重（一日内更新了多次的）
NCP_data = df.drop_duplicates(
    subset=['provinceName', 'updateTime'], keep='first', inplace=False)

print(df.shape)
print(NCP_data.shape)

(87840, 14)
(4542, 14)


In [89]:
# 获取日期list
date_A = []
date = list(set(NCP_data['updateTime']))
for i in date:
    lem_a = date.index(i)
    if (lem_a%2) == 0:
        date_A.append(i)

date = date_A

In [90]:
confirmed_date = {}
province_percent = {}
total_num = []

MapData = []
dict = {}
data = []
dict2 = {}
# list = []
tmpProvinceList = []
rdate = list(date)
rdate.reverse()

In [91]:
for i in date:
    # print(i)
    criteria = NCP_data['updateTime'] == i
    df = NCP_data[criteria]

    for index in df.index:
        tmpProvinceList.append(df.loc[index, 'provinceName'])
    qlist = list(set(provinceName) - (set(tmpProvinceList)))
    # 缺失省份的名单
    print(qlist)
    old_suspectedCount = 0
    old_curedCount = 0
    old_deadCount = 0
    old_confirmedCount = 0
    isChanged = False
    for q in qlist:
        for td in rdate[:rdate.index(i)]:
            criteria = NCP_data['updateTime'] == td
            tddf = NCP_data[criteria]
            for j in tddf.index:
                if str(tddf.loc[j, 'provinceName']) == q:
                    old_confirmedCount = int(
                        tddf.loc[j, 'province_confirmedCount'])
                    old_curedCount = int(tddf.loc[j, 'province_curedCount'])
                    old_deadCount = int(tddf.loc[j, 'province_deadCount'])
                    old_suspectedCount = int(
                        tddf.loc[j, 'province_suspectedCount'])
                    isChanged = True
                    break
        if isChanged:
            NCP_data = NCP_data.append({'provinceName': q,
                                        'province_suspectedCount': old_suspectedCount,
                                        'province_curedCount': old_curedCount,
                                        'province_deadCount': old_deadCount,
                                        'province_confirmedCount': old_confirmedCount,
                                        'updateTime': i
                                        }, ignore_index=True)
            isChanged = False

    tmpProvinceList.clear()

    all_cases = df['province_confirmedCount'].sum()
    confirmed_date[i] = all_cases
    total_num.append(all_cases / 100)
    total_num.sort()

['江苏', '西藏', '青海', '陕西', '内蒙古', '新疆', '甘肃']
[]
[]
['西藏', '青海']
[]
[]
['西藏', '内蒙古']
['西藏', '青海', '甘肃']
['吉林', '西藏', '福建', '天津', '青海', '山西', '云南', '河北', '宁夏']
['西藏', '贵州']
[]
['西藏', '青海']
[]
[]
['西藏']
[]
['西藏', '福建', '青海', '云南', '内蒙古', '新疆', '宁夏', '甘肃', '安徽']
['西藏']
[]
[]
['西藏']
[]
[]
[]
['西藏']
[]
['海南', '吉林', '河南', '西藏', '湖南', '重庆', '山西', '云南', '江苏', '江西', '贵州', '青海', '内蒙古', '河北', '新疆', '宁夏', '安徽']
[]
['西藏', '贵州', '甘肃', '青海']
['西藏']


In [92]:
NCP_data.reset_index(drop=True)

isGotGuangDong = False
isGotHuBei = False
maxNum = 0
maxCount = 0

In [93]:
for i in date:
    criteria = NCP_data['updateTime'] == i
    df = NCP_data[criteria]
    df['percent'] = df['province_confirmedCount'] / confirmed_date[i]
    #print(df.shape)
    for index in df.index:
        data.append({'name': df.loc[index, 'provinceName'], 'value': [int(
            df.loc[index, 'province_confirmedCount']), float(df.loc[index, 'percent']),
            str(df.loc[index, 'provinceName'])]})
        if not isGotGuangDong and df.loc[index, 'provinceName'] == '广东':
            maxNum = int(df.loc[index, 'province_confirmedCount'])
            isGotGuangDong = True
        if not isGotHuBei and df.loc[index, 'provinceName'] == '湖北':
            maxCount = int(df.loc[index, 'province_confirmedCount'])
            isGotHuBei = True
    data = sorted(data, key=lambda x: -x['value'][1])
    MapData.append({'time': i, 'data': list(data)})
    data.clear()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [94]:
def Reverse(lst):
    return [ele for ele in reversed(lst)]

In [95]:
time_list = Reverse(date)
# print(time_list)

fout = open('detail_content.json', 'w', encoding='utf8')
fout.write(str(MapData))
fout.close()

minNum = maxNum / 25

In [96]:
def get_year_chart(year: str):
    map_data = [
        [[x["name"], x["value"]] for x in d["data"]] for d in MapData if d["time"] == year
    ][0]
    min_data, max_data = (minNum, maxNum)
    data_mark: List = []
    i = 0
    for x in time_list:
        if x == year:
            data_mark.append(total_num[i])
        else:
            data_mark.append("")
        i = i + 1

    map_chart = (
        Map()
        .add(
            series_name="",
            data_pair=map_data,
            zoom=1,
            center=[119.5, 34.5],
            is_map_symbol_show=False,
            itemstyle_opts={
                "normal": {"areaColor": "#323c48", "borderColor": "#404a59"},
                "emphasis": {
                    "label": {"show": Timeline},
                    "areaColor": "rgba(255,255,255, 0.5)",
                },
            },
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="" +
                      str(year) + "全国各省份NCP实时动态(数据来源:丁香园; 数据仓库:BlankerL/DXY-2019-nCoV-Data)",
                subtitle="",
                pos_left="center",
                pos_top="top",
                title_textstyle_opts=opts.TextStyleOpts(
                    font_size=25, color="rgba(255,255,255, 0.9)"
                      ),
            ),
            tooltip_opts=opts.TooltipOpts(
                is_show=True,
                formatter=JsCode(
                    """function(params) {
                    if ('value' in params.data) {
                        return params.data.value[2] + ': ' + params.data.value[0];
                    }
                }"""
                ),
            ),
            visualmap_opts=opts.VisualMapOpts(
                is_calculable=True,
                dimension=0,
                pos_left="30",
                pos_top="center",
                range_text=["High", "Low"],
                range_color=["lightskyblue", "yellow", "orangered"],
                textstyle_opts=opts.TextStyleOpts(color="#ddd"),
                min_=min_data,
                max_=max_data,
            ),
        )
    )

    line_chart = (
        Line()
        .add_xaxis(time_list)
        .add_yaxis("", total_num)
        .add_yaxis(
            "",
            data_mark,
            markpoint_opts=opts.MarkPointOpts(
                data=[opts.MarkPointItem(type_="max")]),
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="全国各省份NCP实时动态(单位: 百人)", pos_left="72%", pos_top="5%"
            )
        )
    )
    bar_x_data = [x[0] for x in map_data]
    bar_y_data = [{"name": x[0], "value": x[1][0]} for x in map_data]
    bar = (
        Bar()
        .add_xaxis(xaxis_data=bar_x_data)
        .add_yaxis(
            series_name="",
            yaxis_data=bar_y_data,
            label_opts=opts.LabelOpts(
                is_show=True, position="right", formatter="{b} : {c}"
            ),
        )
        .reversal_axis()
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                max_=maxCount, axislabel_opts=opts.LabelOpts(is_show=False)
            ),
            yaxis_opts=opts.AxisOpts(
                axislabel_opts=opts.LabelOpts(is_show=False)),
            tooltip_opts=opts.TooltipOpts(is_show=False),
            visualmap_opts=opts.VisualMapOpts(
                is_calculable=True,
                dimension=0,
                pos_left="10",
                pos_top="top",
                range_text=["High", "Low"],
                range_color=["lightskyblue", "yellow", "orangered"],
                textstyle_opts=opts.TextStyleOpts(color="#ddd"),
                min_=min_data,
                max_=max_data,
            ),
        )
    )

    pie_data = [[x[0], x[1][0]] for x in map_data]
    pie = (
        Pie()
        .add(
            series_name="",
            data_pair=pie_data,
            radius=["15%", "35%"],
            center=["80%", "82%"],
            itemstyle_opts=opts.ItemStyleOpts(
                border_width=1, border_color="rgba(0,0,0,0.5)"
            ),
        )
        .set_global_opts(
            tooltip_opts=opts.TooltipOpts(is_show=True, formatter="{b} {d}%"),
            legend_opts=opts.LegendOpts(is_show=False),
        )
    )

    grid_chart = (
        Grid()
        .add(
            bar,
            grid_opts=opts.GridOpts(
                pos_left="10", pos_right="45%", pos_top="50%", pos_bottom="5"
            ),
        )
        .add(
            line_chart,
            grid_opts=opts.GridOpts(
                pos_left="65%", pos_right="80", pos_top="10%", pos_bottom="50%"
            ),
        )
        .add(pie, grid_opts=opts.GridOpts(pos_left="45%", pos_top="60%"))
        .add(map_chart, grid_opts=opts.GridOpts())
    )

    return grid_chart


In [97]:
def getDataByDate(date):
    criteria = NCP_data['updateTime'] == date
    return NCP_data[criteria]

In [98]:
df = getDataByDate('03-21')

In [99]:
df.head()

,continentName,continentEnglishName,countryName,countryEnglishName,provinceName,provinceEnglishName,province_zipCode,province_confirmedCount,province_suspectedCount,province_curedCount,province_deadCount,updateTime,cityEnglishName,city_zipCode
0,亚洲,Asia,中国,China,台湾,Taiwan,710000.0,153,0.0,28,2,03-21,NaN,NaN
1,亚洲,Asia,中国,China,北京,Beijing,110000.0,504,0.0,396,8,03-21,NaN,0.0
2,亚洲,Asia,中国,China,广东,Guangdong,440000.0,1400,0.0,1325,8,03-21,Guangzhou,440100.0
3,亚洲,Asia,中国,China,上海,Shanghai,310000.0,380,0.0,327,3,03-21,NaN,0.0
4,欧洲,Europe,意大利,Italy,意大,Italy,965008.0,47021,0.0,5129,4032,03-21,NaN,NaN


In [100]:
def getDataByCountry(country):
    criteria = NCP_data['countryName'] == country
    return NCP_data[criteria]

In [101]:
getDataByCountry('意大利')

,continentName,continentEnglishName,countryName,countryEnglishName,provinceName,provinceEnglishName,province_zipCode,province_confirmedCount,province_suspectedCount,province_curedCount,province_deadCount,updateTime,cityEnglishName,city_zipCode
4,欧洲,Europe,意大利,Italy,意大,Italy,965008.0,47021,0.0,5129,4032,03-21,NaN,NaN
186,欧洲,Europe,意大利,Italy,意大,Italy,965008.0,41035,0.0,4440,3405,03-20,NaN,NaN
366,欧洲,Europe,意大利,Italy,意大,Italy,965008.0,37178,0.0,4025,2978,03-19,NaN,NaN
621,欧洲,Europe,意大利,Italy,意大,Italy,965008.0,31506,0.0,2941,2503,03-18,NaN,NaN
716,欧洲,Europe,意大利,Italy,意大,Italy,965008.0,29022,0.0,2749,2158,03-17,NaN,NaN
882,欧洲,Europe,意大利,Italy,意大,Italy,965008.0,24938,0.0,2335,1809,03-16,NaN,NaN
1185,欧洲,Europe,意大利,Italy,意大,Italy,965008.0,21270,0.0,1966,1441,03-15,NaN,NaN
1257,欧洲,Europe,意大利,Italy,意大,Italy,965008.0,17660,0.0,1439,1266,03-14,NaN,NaN
1398,欧洲,Europe,意大利,Italy,意大,Italy,965008.0,15113,0.0,1258,1016,03-13,NaN,NaN
1517,欧洲,Europe,意大利,Italy,意大,Italy,965008.0,12462,0.0,1045,827,03-12,NaN,NaN


In [102]:
getDataByCountry('美国')

,continentName,continentEnglishName,countryName,countryEnglishName,provinceName,provinceEnglishName,province_zipCode,province_confirmedCount,province_suspectedCount,province_curedCount,province_deadCount,updateTime,cityEnglishName,city_zipCode
7,北美洲,North America,美国,United States of America,美国,United States of America,971002.0,19624,0.0,147,260,03-21,NaN,NaN
189,北美洲,North America,美国,United States of America,美国,United States of America,971002.0,14250,0.0,121,208,03-20,NaN,NaN
370,北美洲,North America,美国,United States of America,美国,United States of America,971002.0,10259,0.0,106,153,03-19,NaN,NaN
623,北美洲,North America,美国,United States of America,美国,United States of America,971002.0,6420,0.0,74,108,03-18,NaN,NaN
724,北美洲,North America,美国,United States of America,美国,United States of America,971002.0,4661,0.0,48,90,03-17,NaN,NaN
909,北美洲,North America,美国,United States of America,美国,United States of America,971002.0,3700,0.0,10,66,03-16,NaN,NaN
1046,北美洲,North America,美国,United States of America,美国,United States of America,971002.0,2885,0.0,10,60,03-15,NaN,NaN
1220,北美洲,North America,美国,United States of America,美国,United States of America,971002.0,2084,0.0,10,41,03-14,NaN,NaN
1340,北美洲,North America,美国,United States of America,美国,United States of America,971002.0,1635,0.0,10,39,03-13,NaN,NaN
1519,北美洲,North America,美国,United States of America,美国,United States of America,971002.0,1004,0.0,10,31,03-12,NaN,NaN


In [81]:
NCP_data.iloc[0, ]

continentName                  亚洲
continentEnglishName         Asia
countryName                    中国
countryEnglishName          China
provinceName                   台湾
provinceEnglishName        Taiwan
province_zipCode           710000
province_confirmedCount       153
province_suspectedCount         0
province_curedCount            28
province_deadCount              2
updateTime                  03-21
cityEnglishName               NaN
city_zipCode                  NaN
Name: 0, dtype: object